In [1]:
import os
os.chdir('/kaggle/working')

In [2]:
!pip install audeer audonnx audiofile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.2 MB/s eta 0:00:00


In [3]:
#Import Dependencies

import audeer
import audonnx
import numpy as np
import pandas as pd
import audiofile
import time
import datetime as dt

cache_root = audeer.mkdir('cache')
model_root = audeer.mkdir('model')

In [4]:
def calcProcessTime(starttime, cur_iter, max_iter):

    telapsed = time.time() - starttime
    testimated = (telapsed/cur_iter)*(max_iter)

    finishtime = starttime + testimated
    finishtime = dt.datetime.fromtimestamp(finishtime).strftime("%H:%M:%S")  # in time

    lefttime = testimated-telapsed  # in seconds

    return (int(telapsed), int(lefttime), finishtime)

In [5]:
#Download model if necessary
url = 'https://zenodo.org/record/6221127/files/w2v2-L-robust-12.6bc4a7fd-1.1.0.zip'

archive_path = audeer.download_url(url, cache_root, verbose=True)
audeer.extract_archive(archive_path, model_root)

['model.onnx', 'model.yaml']

In [6]:
#Load model and test with random sample
model = audonnx.load(model_root)
sampling_rate = 16000
signal = np.random.normal(size=sampling_rate).astype(np.float32)
model(signal, sampling_rate)

{'hidden_states': array([[-0.00714738,  0.00631136, -0.00733255, ...,  0.00666412,
          0.00946707,  0.00295012]], dtype=float32),
 'logits': array([[0.67524654, 0.65099585, 0.5139543 ]], dtype=float32)}

In [7]:
#Function definitions to use the model to extract arousal valence dominance from an audiofile

def predict_avd(path):
    signal, sampling_rate = audiofile.read(
                path,
                always_2d=True,
            )
    ret = model(signal, sampling_rate)
    return ret["logits"][0]
start = time.time()

def extract_avd(input_file,output_file):
    df = pd.read_csv(input_file)
    # df = df.sample(n=3) # test on small subset
    data = {"ActorID":[],"Emotion":[],"Path":[],"arousal":[],"valence":[],"dominance":[]}
    cur_iter = 0
    for index, row in df.iterrows():
        ret = predict_avd(row["Path"])
        data["arousal"].append(ret[0])
        data["valence"].append(ret[1])
        data["dominance"].append(ret[2])
        data["Path"].append(row['Path'])
        data["Emotion"].append(row['Emotion'])
        data["ActorID"].append(row['ActorID'])
        cur_iter += 1
        print("On {}%".format(round(cur_iter / df.shape[0] * 100, 2)))
        prstime = calcProcessTime(start, cur_iter ,df.shape[0])
        print("time elapsed: %s(s), time left: %s(s), estimated finish time: %s"%prstime)
    resdf = pd.DataFrame.from_dict(data)
    resdf.to_csv(output_file,index=False)


In [8]:
extract_avd('/kaggle/input/opensmile-beasc/opensmileDF_beasc.csv', "/kaggle/working/beasc_avd.csv")

On 0.08%
time elapsed: 9(s), time left: 11802(s), estimated finish time: 00:51:49
On 0.16%
time elapsed: 18(s), time left: 11338(s), estimated finish time: 00:44:14
On 0.25%
time elapsed: 25(s), time left: 10326(s), estimated finish time: 00:27:29
On 0.33%
time elapsed: 35(s), time left: 10720(s), estimated finish time: 00:34:13
On 0.41%
time elapsed: 42(s), time left: 10245(s), estimated finish time: 00:26:25
On 0.49%
time elapsed: 47(s), time left: 9600(s), estimated finish time: 00:15:45
On 0.57%
time elapsed: 56(s), time left: 9801(s), estimated finish time: 00:19:15
On 0.65%
time elapsed: 65(s), time left: 10027(s), estimated finish time: 00:23:10
On 0.74%
time elapsed: 74(s), time left: 10072(s), estimated finish time: 00:24:04
On 0.82%
time elapsed: 84(s), time left: 10242(s), estimated finish time: 00:27:03
On 0.9%
time elapsed: 93(s), time left: 10291(s), estimated finish time: 00:28:01
On 0.98%
time elapsed: 102(s), time left: 10397(s), estimated finish time: 00:29:58
On 1.06

In [9]:
def feature_embeddings(path):
    signal, sampling_rate = audiofile.read(
                path,
                always_2d=True,
            )
    ret = model(signal, sampling_rate)
    return ret["hidden_states"][0]

In [10]:
from tqdm import tqdm

def extract_transformer_embeddings(input_file, output_file):
    start = time.time()
    df = pd.read_csv(input_file)
    # df = df.sample(n=3)  # test on small subset
    data = {"Path": [], "feature_embeddings": []}
    
    cur_iter = 0
    # Wrap the iterable with tqdm for the progress bar
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        ret = feature_embeddings(row["Path"])
        data["Path"].append(row['Path'])
        data["feature_embeddings"].append(list(ret))
        cur_iter += 1
        print("On {}%".format(round(cur_iter / df.shape[0] * 100, 2)))
        prstime = calcProcessTime(start, cur_iter ,df.shape[0])
        print("time elapsed: %s(s), time left: %s(s), estimated finish time: %s"%prstime)
    
    resdf = pd.DataFrame.from_dict(data)
    resdf.to_csv(output_file, index=False)
        

In [11]:
extract_transformer_embeddings("/kaggle/input/opensmile-beasc/opensmileDF_beasc.csv","/kaggle/working/beasc_embeddings.csv")

Processing rows:   0%|          | 1/1224 [00:09<3:07:44,  9.21s/it]

On 0.08%
time elapsed: 9(s), time left: 11312(s), estimated finish time: 03:07:54


Processing rows:   0%|          | 2/1224 [00:17<3:01:54,  8.93s/it]

On 0.16%
time elapsed: 17(s), time left: 10989(s), estimated finish time: 03:02:40


Processing rows:   0%|          | 3/1224 [00:24<2:39:48,  7.85s/it]

...

On 99.84%
time elapsed: 8642(s), time left: 14(s), estimated finish time: 02:23:29


Processing rows: 100%|█████████▉| 1223/1224 [2:24:11<00:08,  8.54s/it]

On 99.92%
time elapsed: 8651(s), time left: 7(s), estimated finish time: 02:23:30


Processing rows: 100%|██████████| 1224/1224 [2:24:18<00:00,  7.07s/it]

On 100.0%
time elapsed: 8658(s), time left: 0(s), estimated finish time: 02:23:31


In [12]:
df = pd.read_csv("/kaggle/working/beasc_embeddings.csv")
df2 = pd.read_csv("/kaggle/input/opensmile-beasc/opensmileDF_beasc.csv")

df3 = df.merge(df2, how="left",on=["Path"])

In [13]:
df3.to_csv('/kaggle/working/merged_beasc_df_embeddings')